In [1]:
from ct_model import DeltaModel, dispatch_default
from ct_bnb import Linear4bit, dispatch_bnb_4bit
from ct_layer import DeltaLayer
from ct_config import CTConfig
from ct_optim import BlockOptimizer
from prepare_data import gen_dataloader

from torch.optim import AdamW
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

from tqdm import tqdm

/home/ubuntu/date/mq_tst/anaconda3/envs/newpeft/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
device_id = 2

# Quantization type (fp4 or nf4), According to QLoRA paper, for training 4-bit base models (e.g. using LoRA adapters) one should use
bnb_4bit_quant_type = "fp4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True

model_id = "Qwen/Qwen1.5-0.5B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=use_nested_quant,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":device_id})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
config = CTConfig(
             r=32,
             delta_alpha=32,
             target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                             "gate_proj", "up_proj", "down_proj"],
             delta_dropout=0,
             init_lora_weights=False,
         )

In [4]:
# config = CTConfig(
#              r=32,
#              delta_alpha=32,
#              target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
#              delta_dropout=0,
#              init_lora_weights=False,
#          )

In [5]:
delta_model = DeltaModel(model, config, "default")

In [6]:
base_optimizer = AdamW(model.parameters(), lr=1e-4)

In [7]:
optimizer = BlockOptimizer(base_optimizer,
                           list(delta_model.named_parameters()),
                           None,
                           10,
                           switch_mode="ascending",
                           model=delta_model)

Now init the block adapter ['model.model.layers.0.']
Parameters with the following prefix will be trainable: ['model.model.layers.0.']


In [8]:
train_dataloader = gen_dataloader(tokenizer)

Using the latest cached version of the module from /home/ubuntu/date/hf_cache/modules/datasets_modules/datasets/eli5_category/80106cc49322f1f5075e1387be4a5b74b95e0f56c40ff142b8999d0606aa1908 (last modified on Wed Jun  5 22:09:48 2024) since it couldn't be found locally at eli5_category, or remotely on the Hugging Face Hub.


Map (num_proc=4):   0%|          | 0/800 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/200 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/800 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/200 [00:00<?, ? examples/s]

In [9]:
for step, batch in enumerate(tqdm(train_dataloader)):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]

    outputs = delta_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss

    if step % 5 ==0:
        print(loss)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  0%|          | 1/800 [00:00<12:30,  1.07it/s]

tensor(3.1361, grad_fn=<ToCopyBackward0>)


  1%|          | 6/800 [00:01<03:08,  4.20it/s]

tensor(4.5185, grad_fn=<ToCopyBackward0>)


  1%|          | 8/800 [00:02<02:50,  4.64it/s]

...

tensor(4.2594, grad_fn=<ToCopyBackward0>)


 13%|█▎        | 106/800 [00:30<02:28,  4.67it/s]

tensor(3.1296, grad_fn=<ToCopyBackward0>)


 13%|█▎        | 107/800 [00:30<03:19,  3.48it/s]


KeyboardInterrupt: 

In [10]:
delta_model

DeltaModel(
  (model): Qwen2ForCausalLM(
    (model): Qwen2Model(
      (embed_tokens): Embedding(151936, 1024)
      (layers): ModuleList(
        (0-9): 10 x Qwen2DecoderLayer(
          (self_attn): Qwen2SdpaAttention(
            (q_proj): lora.Linear4bit(
              (base_layer): Linear4bit(in_features=1024, out_features=1024, bias=True)
              (delta_dropout): ModuleDict(
                (default): Identity()
              )
              (delta_theta): ModuleDict()
              (delta_A): ModuleDict(
                (default): Linear(in_features=1024, out_features=32, bias=True)
              )
              (delta_B): ModuleDict(
                (default): Linear(in_features=32, out_features=1024, bias=False)
              )
              (delta_embedding): ParameterDict()
            )
            (k_proj): lora.Linear4bit(
              (base_layer): Linear4bit(in_features=1024, out_features=1024, bias=True)
              (delta_dropout): ModuleDict(
              